In [1]:
%load_ext autoreload
%autoreload 2

import os

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import LineString
from shapely import wkt
import numpy as np
import swifter
from generator.preprocess import *
from generator.preprocess import remove_outlier_trajectories
from generator.road_network import RoadNetwork
import glob
from tqdm import tqdm

In [2]:
df = pd.read_csv("../datasets/trajectories/hanover/hannover_inrix.csv")
network = RoadNetwork()
network.load("../osm_data/hanover")

In [3]:
l, c = 0, 0
for _, g in tqdm(df.groupby("id")):
    if g.shape[0] < 5:
        continue
    l += g.shape[0]
    c += 1

print(l/c)


100%|██████████| 77188/77188 [00:02<00:00, 34545.13it/s]

241.32424353095539


In [3]:
rows = []
for _, g in tqdm(df.groupby("id")):
    if g.shape[0] < 5:
        continue
    traj = g[["longitude", "latitude", "time"]].copy()
    traj.loc[:, "time"] = pd.to_datetime(traj["time"], infer_datetime_format=True).dt.floor('s')
    traj.loc[:, "time"] = traj["time"].values.astype(np.int64) // 10 ** 9
    
    data = traj.to_numpy()
    time = data[1:, 2] - data[0, 2]
    time = np.insert(time, 0, 0)
    seq = LineString(data[:, :2])
    rows.append((seq, time.astype(int).tolist(), traj.iloc[0, 2], traj.iloc[-1, 2]))

processed_df = pd.DataFrame(rows, columns=["POLYLINE", "timestamp", "start_stamp", "end_stamp"])
processed_df

100%|██████████| 77188/77188 [03:20<00:00, 384.35it/s]


,POLYLINE,timestamp,start_stamp,end_stamp
0,"LINESTRING (9.8288 52.3326, 9.819705 52.331356...","[0, 80, 424, 535, 613, 1094, 1171, 1453, 1528,...",1565974403,1565982018
1,"LINESTRING (9.6472 52.2823, 9.643848 52.283978...","[0, 30, 60, 90, 120, 172, 202, 232, 262, 292, ...",1567075203,1567075868
2,"LINESTRING (9.7376 52.5465, 9.72797 52.541855,...","[0, 138, 300, 485, 668, 801, 882]",1566184436,1566185318
3,"LINESTRING (9.7119 52.343, 9.716048 52.344969,...","[0, 90, 180, 229, 319, 409, 492, 582, 672, 762]",1568621637,1568622399
4,"LINESTRING (9.7585 52.3363, 9.762929 52.34235,...","[0, 79, 141, 224, 377, 453, 580, 685, 763, 881...",1566671810,1566673656
...,...,...,...,...
73924,"LINESTRING (10.0369 53.3593, 10.037012 53.3582...","[0, 5, 8, 14, 15, 19, 25, 28, 30, 34, 39, 44, ...",1569563606,1569569649
73925,"LINESTRING (9.7278 52.3654, 9.727219 52.365731...","[0, 5, 8, 16, 19, 20, 25, 29, 35, 40, 46, 49, ...",1575029942,1575031187
73926,"LINESTRING (9.8726 52.9458, 9.718267 52.63466,...","[0, 599, 703, 718, 733, 748, 763, 778, 1295, 1...",1573224957,1573226965
73927,"LINESTRING (9.738963 52.382663, 9.738996 52.38...","[0, 5, 10, 15, 20, 25, 30, 35, 45, 50, 55, 60,...",1574319815,1574320945


In [51]:
city_bounds = network.bounds
df_clipped = clip_trajectories(processed_df.copy(), city_bounds, polyline_convert=True)
df_clipped = filter_min_points(df_clipped, 5)

(73928, 0)

In [9]:
df_clipped["id"] = np.arange(1, df_clipped.shape[0]+1)
df_clipped.drop("coords", inplace=True, axis=1)
df_clipped["timestamp"] = df_clipped["timestamp"].astype(str)
df_clipped["timestamp"] = df_clipped["timestamp"].str.replace("[", "")
df_clipped["timestamp"] = df_clipped["timestamp"].str.replace("]", "")
#df_clipped["timestamp"] = df_clipped["timestamp"].str.replace("  ", ", ")
df_clipped.to_csv("../datasets/trajectories/hanover/mapped_id_poly_clipped.csv", sep=";", index=False)

In [10]:
df = pd.read_csv("../datasets/trajectories/hanover/mapped_id_poly_clipped.csv", sep=";")
df

,timestamp,start_stamp,end_stamp,POLYLINE,id
0,"0, 80, 424, 535, 613, 1094, 1171, 1453, 1528, ...",1565974403,1565982018,"LINESTRING (9.8288 52.3326, 9.819705 52.331356...",1
1,"0, 30, 60, 90, 120, 172, 202, 232, 262, 292, 5...",1567075203,1567075868,"LINESTRING (9.660448261338471 52.3136182, 9.66...",2
2,"0, 90, 180, 229, 319, 409, 492, 582, 672, 762",1568621637,1568622399,"LINESTRING (9.7119 52.343, 9.716048 52.344969,...",3
3,"0, 79, 141, 224, 377, 453, 580, 685, 763, 881,...",1566671810,1566673656,"LINESTRING (9.7585 52.3363, 9.762929 52.34235,...",4
4,"0, 1, 9, 15, 26, 30, 34, 41, 46, 51, 60, 66, 7...",1574157039,1574159903,"LINESTRING (9.6138059 52.428522640720765, 9.62...",5
...,...,...,...,...,...
143854,"0, 5, 8, 14, 15, 19, 25, 28, 30, 34, 39, 44, 4...",1569563606,1569569649,"LINESTRING (9.718658264790285 52.4474529, 9.71...",143855
143855,"0, 5, 8, 16, 19, 20, 25, 29, 35, 40, 46, 49, 5...",1575029942,1575031187,"LINESTRING (9.7278 52.3654, 9.727219 52.365731...",143856
143856,"0, 599, 703, 718, 733, 748, 763, 778, 1295, 16...",1573224957,1573226965,"LINESTRING (9.782460425078042 52.4474529, 9.78...",143857
143857,"0, 5, 10, 15, 20, 25, 30, 35, 45, 50, 55, 60, ...",1574319815,1574320945,"LINESTRING (9.738963 52.382663, 9.738996 52.38...",143858


In [6]:
df["POLYLINE"] = df["POLYLINE"].swifter.apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs="epsg:4326", geometry="POLYLINE")

Pandas Apply:   0%|          | 0/153576 [00:00<?, ?it/s]

In [7]:
network.fmm_trajectorie_mapping(
    
    network_file="../osm_data/hanover/edges.shp",
    input_file="../datasets/trajectories/Hanover/mapped_id_poly_clipped.csv",
    output_file="../datasets/trajectories/Hanover/road-segment-mapping.txt"
)

In [30]:
# preprocess the mapping especially the speed and distance values need to be verified
df = pd.read_csv("../datasets/trajectories/hanover/road-segment-mapping.csv", sep=";")
df_prep = remove_outlier_trajectories(df.copy(), min_edges_traversed=3, max_speed=0.000251)
df_prep.to_csv("../datasets/trajectories/hanover/road_segment_map_final.csv", sep=";")

Pandas Apply:   0%|          | 0/82406 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/82406 [00:00<?, ?it/s]

/dstore/home/pheinemeyer/Road-Network-Embedding-Generator/generator/preprocess.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[(df["speed_mean"] > max_speed)]["speed"] = df[(df["speed_mean"] > max_speed)][


Pandas Apply:   0%|          | 0/56921 [00:00<?, ?it/s]

/dstore/home/pheinemeyer/Road-Network-Embedding-Generator/generator/preprocess.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["speed_mean"] = df["speed"].swifter.apply(np.mean)


In [13]:
"""
Test of Travel Time Dataset generation
"""
from generator.trajectory import Trajectory

traj = Trajectory("../datasets/trajectories/hanover/road_segment_map_final.csv", nrows=1000000)

Pandas Apply:   0%|          | 0/42357 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/42357 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/42357 [00:00<?, ?it/s]

In [14]:
"""
Generate traj features 
"""
features = traj.generate_speed_features(network)
features.to_csv("../datasets/trajectories/hanover/speed_features_unnormalized.csv")

42357it [00:21, 1989.20it/s]


In [15]:
features.describe()

,id,util,avg_speed
count,18576.000000,18576.000000,13751.000000
mean,9287.500000,42.012220,38.970848
std,5362.573636,106.154144,20.793826
min,0.000000,0.000000,0.000000
25%,4643.750000,1.000000,25.955203
50%,9287.500000,5.000000,37.289088
75%,13931.250000,31.000000,48.714460
max,18575.000000,1712.000000,194.398395


In [16]:
print(traj.df.shape)

(42357, 10)
